#  Modelación Volatilidad en Tasas de Cambio con modelos de la familia GARCH

**Objetivo**: El objetivo de este Notebook es implementar modelos clásicos de volatilidad como GARCH (Generalized ARCH Models), y EGARCH (Exponential GARCH) y GJR para predecir la tasa de cambio entre monedas.

**Descripción Dataset**: El dataset utilizado contiene la información histórica entre "2021-04-06" y "2024-04-06" con frecuencia diaria para las siguientes tasas de cambio:
- `Dolar EstadoUnidense` (USD) contra `Dolar Canadiende` (CAD)
- `Dolar EstadoUnidense` (USD) contra `Dolar Australiano` (CAD)

**Métricas de evaluación de modelos**: 
- `Mean Squared Error (MSE)`
- `Mean Absolute Error (MAE)`
- `Mean Absolute Percentage Error (MAPE)`

**Integrantes**
- David López
- Camilo Velez
- Sebastian Ávila
- David Armendariz

## Importando Librerías

In [7]:
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import het_arch
from statsmodels.compat import lzip
import statsmodels.api as sm
import pandas as pd
import numpy as np
from arch import arch_model

## Cargando DataSet

In [5]:
df=pd.read_csv("../data/trusted/currency_exchange.csv")
print(df.dtypes)
print(df.shape)
display(df.head())
display(df.tail())

Fecha       object
USD_CAD    float64
USD_AUD    float64
dtype: object
(1096, 3)


,Fecha,USD_CAD,USD_AUD
0,2021-04-06,1.2566,1.3049
1,2021-04-07,1.2609,1.3137
2,2021-04-08,1.2562,1.3066
3,2021-04-09,1.2530,1.3125
4,2021-04-10,1.2530,1.3125


,Fecha,USD_CAD,USD_AUD
1091,2024-04-01,1.3570,1.5410
1092,2024-04-02,1.3567,1.5342
1093,2024-04-03,1.3527,1.5233
1094,2024-04-04,1.3543,1.5180
1095,2024-04-05,1.3590,1.5201


## Testeando ARCH